In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<font size="+3" color=red><b> <center><u> Fantasy IPL Dream Team🏏 for whole tournament and season 2020 </u></center></b></font>

In [ ]:
df1=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df1.head()

In [ ]:
df2=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
df2.head()

In [ ]:
# We need two dataframes one is for whole tournament and other is for year 2020
# Merge df1 and df2(onle two columns(id,date)) on 'id'
data = pd.merge(df2, df1[['id','date']], on='id',how='left')

# Extract year from date column
data['year'] = pd.to_datetime(data['date']).dt.year
data.drop('date',axis=1,inplace=True)

# This dataframe is only for year 2020
season_2020 = data[data['year']==2020]
# This is basically copy of df2 because it contains data of whole tournament
tournament = df2.copy()

# ****Batting Points****

 **Run                       -> +1**
 
 **Boundary Bonus            -> +1**
 
 **Six Bonus                 -> +2**
 
 **30 Run Bonus              -> +4**
 
 **Half-century Bonus        -> +8**
 
 **Century Bonus             -> +16**

In [ ]:
# It finds bat_score according to point system mentioned above.
def batsman_score(x):
    x["bat_score"]=1*x["runs"]+1*x["fours"]+2*x["sixes"]+4*x["thirty"]+8*x["fifty"]+16*x["century"]
    return x

def get_bat_stats(df):
    # Initialize batting dataframe.
    batting_df = pd.DataFrame(columns=["player","runs","fours","sixes","thirty","fifty","century"])

    # Let's use groupby function. Our aim is to find batting score so we'll group our data by batsman.
    for player,details in df.groupby("batsman"):
        total_runs = sum(details["batsman_runs"])

        # If batsman_runs is 4 then we count it as four.
        fours=0
        try:
            fours=details["batsman_runs"].value_counts()[4]
        except:
            fours=0      

        # If batsman_runs is 6 then we count it as six.
        sixes=0
        try:
            sixes=details["batsman_runs"].value_counts()[6]
        except:
            sixes=0 

        thirty=0
        fifty=0
        century=0

        # To find thirty, fifty, century of the player we need to group our dataframe(details) by match id.
        for match,match_details in details.groupby("id"):

            # Find total score of the player by sum up total runs.
            match_score = sum(match_details["batsman_runs"])

            # Accourding to match_score this count thirties, fifties, centuries.
            if match_score>=100:
                century+=1
            elif match_score>=50:
                fifty+=1
            elif match_score>=30:
                thirty+=1

        # Store all the data into batting dataframe.
        batting_df = batting_df.append({"player":player,"runs":total_runs,"fours":fours,"sixes":sixes,"thirty":thirty,"fifty":fifty,"century":century},ignore_index=True)
    
    # To find bat_score we apply batsman_score function
    batting_df=batting_df.apply(batsman_score,axis=1)
    return batting_df

# ****Bowling Points****

 **Wicket                  -> +25**
 
 **Bonus (LBW / Bowled)    -> +8**
 
 **3 Wicket Bonus          -> +4**
 
 **4 Wicket Bonus          -> +8**
 
 **5 Wicket Bonus          -> +16**

In [ ]:
# Same as bat_score here we find bowl_score through point system mentioned above.
def bowler_score(x):
    x["bowl_score"]=25*x["wickets"]+8*x["lbw_or_bowled"]+4*x["three_wickets"]+8*x["four_wickets"]+16*x["five_wickets"]
    return x

def get_bowl_stats(df):
    # Initialize bowling dataframe.
    bowling_df = pd.DataFrame(columns=["player","wickets","lbw_or_bowled","three_wickets","four_wickets","five_wickets"])

    # Let's use groupby function. Our aim is to find bowling score so we'll group our data by bowler.
    for player,details in df.groupby("bowler"):

        # It finds the length of dataframe where dismissal kind is either lbw or bowled
        lbw_or_bowled=len(details.query('dismissal_kind == "lbw" or dismissal_kind == "bowled"'))

        total_wickets=0
        three_wickets=0
        four_wickets=0
        fifer=0

        # To find three wicket haul, four wicket haul and five wicket haul we should group dataframe(details) again by match id.
        for match,match_details in details.groupby("id"):
            wickets = len(match_details.query('dismissal_kind in ["caught","bowled","lbw","stumped","caught and bowled"]'))
            if wickets>=3:
                three_wickets+=1
            elif wickets>=4:
                four_wickets+=1
            elif wickets>=5:
                fifer+=1
            # Need to add every match wickets into total_wickets to find total wickets of bowler.
            total_wickets+=wickets

        # Store all the data into bowling dataframe.
        bowling_df = bowling_df.append({"player":player,"wickets":total_wickets,"lbw_or_bowled":lbw_or_bowled,"three_wickets":three_wickets,"four_wickets":four_wickets,"five_wickets":fifer},ignore_index=True)
        
    # To find bowl_score we apply bowler_score function.
    bowling_df=bowling_df.apply(bowler_score,axis=1) 
    return bowling_df

In [ ]:
def get_wicketkeepers(df):
    # Let's find out wicketkeepers from this dataset. To achieve this we'll check dismissal kind, if it is stumpout then store such fielders name into list
    return list(df.query('dismissal_kind == "stumped"')['fielder'].unique())

**Our Dream11 team will have different combinations of players...**

**Batsman (BAT) - 4**

**Wicket Keeper (WK) - 1**

**All Rounder (AR) - 3**

**Bowler (BWL) - 3**

# **For whole Tournament**

In [ ]:
batsmen_tournament = get_bat_stats(tournament)
batsmen_tournament.head()

In [ ]:
bowlers_tournament = get_bowl_stats(tournament)
bowlers_tournament.head()

In [ ]:
merged_data_tournament = pd.merge(batsmen_tournament,bowlers_tournament,on="player",how="outer").fillna(0)
merged_data_tournament.head()

In [ ]:
# Find allrounder score
merged_data_tournament['allrounder_score']=(merged_data_tournament['bat_score']*merged_data_tournament['bowl_score'])/1000
merged_data_tournament.head()

In [ ]:
wicketkeeper_tournament = get_wicketkeepers(tournament)

In [ ]:
# According to score in particular role we sort data and store players name into dictionary. This dictonary contains players name and players role.
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_tournament = {}

playing_XI_tournament['BAT'] = merged_data_tournament.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_tournament['WK'] = merged_data_tournament.query('player in @wicketkeeper_tournament').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_tournament['AR'] = merged_data_tournament.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_tournament['BWL'] = merged_data_tournament.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [ ]:
print('Fantasy dream team for the whole tournament\n')
for player_role,players in playing_XI_tournament.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

# **For Season 2020**

In [ ]:
batsmen_2020 = get_bat_stats(season_2020)
batsmen_2020.head()

In [ ]:
bowlers_2020 = get_bowl_stats(season_2020)
bowlers_2020.head()

In [ ]:
merged_data_2020 = pd.merge(batsmen_2020,bowlers_2020,on="player",how="outer").fillna(0)
merged_data_2020.head()

In [ ]:
# Find allrounder score
merged_data_2020['allrounder_score']=(merged_data_2020['bat_score']*merged_data_2020['bowl_score'])/1000
merged_data_2020.head()

In [ ]:
wicketkeeper_2020 = get_wicketkeepers(season_2020)

In [ ]:
# According to score in particular role we sort data and store players name into dictionary. This dictonary contains players name and players role.
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_2020 = {}

playing_XI_2020['BAT'] = merged_data_2020.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_2020['WK'] = merged_data_2020.query('player in @wicketkeeper_2020').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_2020['AR'] = merged_data_2020.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_2020['BWL'] = merged_data_2020.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [ ]:
print('Fantasy dream team for season 2020\n')
for player_role,players in playing_XI_2020.items():
    for player in players:
        print("{:15} {}".format(player,player_role))